# Data Preparation


In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

sns.set_style("whitegrid")

## Load Data

In [2]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

# load train.csv
data_path = "..\product-inventory"
filename = os.path.join(data_path, "grupo-bimbo-inventory-demand/train.csv.zip")

train = pd.read_csv(filename, 
                 usecols=['Semana', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])

# rename columns
train = train.rename(columns={  'Semana': 'Week_num',
                                'Cliente_ID': 'Client_ID',
                                'Demanda_uni_equil': 'adjusted_demand',
                                'Producto_ID': 'Product_ID'})
# define client-product ID
train['ID'] = train.groupby(['Client_ID', 'Product_ID']).ngroup()
unique_ids = train['ID'].unique()

# Define the fraction of IDs to sample
fraction = 0.01  # sample 1% of the IDs

# Calculate the number of IDs to sample
sample_size = int(len(unique_ids) * fraction)

# Choose a random sample of IDs
sampled_ids = np.random.choice(unique_ids, size=sample_size, replace=False)

# Filter the DataFrame to keep all rows with the sampled IDs
train = train[train['ID'].isin(sampled_ids)]

print(len(train))

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\amirh\AppData\Local\Temp\ipykernel_19872\3118936594.py:4: SyntaxWarning: invalid escape sequence '\p'
  data_path = "..\product-inventory"


740674


In [3]:
# Sort the data frame by ID and Week
train = train.groupby(['ID', 'Week_num'], as_index=False).agg({'adjusted_demand': 'sum'})
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

## Data Imputation Function
This function imputes missing observations based on the firms' demand. In this case, I set all missing observations to zero.

In [4]:
def fillin(df):
    '''
    Input
        df: A dataframe of length at most 7, with column names 'Week_num', 'Client_ID', 'Product_ID', 'adjusted_demand', 'ID',
        where 'ID' is the unique idenifier for client id and product id combinations.  The intended input is train[train['ID' == id]],
        where id is an element of the list train['ID'].unique().

    Outputs
        new_df: If df has 'adjusted_demand' values for each week (3 through 9), new_df = df, i.e. nothing happens.

                If df has missing 'adjusted_demand' values for any week, the 'adjusted_demand' for that week will be 0.
    '''

    # EB: I'm not sure if it matters, but does it need to be a deep copy?
    new_df = df.copy(deep=True).reset_index(drop=True)

    week_list = new_df['Week_num'].unique().tolist()
    missing_week_list = [x for x in [3,4,5,6,7,8,9] if x not in week_list]

    for i in missing_week_list:
        
        #create new row in new_df with the floor of the average value of prev_value and next_value
        new_df = pd.concat([new_df, pd.DataFrame({'Week_num': i,
                                                  'Client_ID': new_df['Client_ID'].iloc[0],
                                                  'Product_ID': new_df['Product_ID'].iloc[0],
                                                  'adjusted_demand': 0,
                                                  'ID': new_df['ID'].iloc[0]}, index=[i])]).sort_values(by=['Week_num']).reset_index(drop=True)
        
        #update week_list
        week_list.append(i)
        
    return new_df

Another way to impute the data is to expand the `train` such as it incluldes all possible ID x Week combination. We'll impute the data a later point.

In [5]:
unq_week = pd.DataFrame({'Week_num': train['Week_num'].unique()})
unq_week = unq_week.sort_values(by='Week_num').reset_index(drop=True)
unq_id = pd.DataFrame({'ID': train['ID'].unique()})
unq_id = unq_id.sort_values(by='ID').reset_index(drop=True)
combo = unq_id.merge(unq_week, how='cross')
train = combo.merge(train, how='outer', on=['ID', 'Week_num'], sort=True)

del combo, unq_week, unq_id
train

,ID,Week_num,adjusted_demand
0,345,3,8.0
1,345,4,9.0
2,345,5,3.0
3,345,6,11.0
4,345,7,12.0
...,...,...,...
1808200,25831381,5,NaN
1808201,25831381,6,NaN
1808202,25831381,7,NaN
1808203,25831381,8,NaN


## Time Series Estimation

### Define Variables
I define new variables and modify the existing ones.

In [6]:
# Define lagged demand and lagged week
train['adj_demand_1'] = train['adjusted_demand'].shift(1)
train['week_1'] = train['Week_num']-1
train['adj_demand_1'] = train['adj_demand_1'].where(train['Week_num'] != 3, np.nan)

# Define log demand and log lagged demand
train['y'] = np.log(train['adjusted_demand'])
train['y'] = train['y'].replace([np.inf, -np.inf], np.nan)
train['y_1'] = np.log(train['adj_demand_1'])
train['y_1'] = train['y_1'].replace([np.inf, -np.inf], np.nan)

c:\Users\amirh\.julia\conda\3\envs\erdos_spring_2025\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amirh\.julia\conda\3\envs\erdos_spring_2025\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Models
1. $demand_{t}$ on $demand_{t-1}$
2. $log(demand_{t})$ on $log(demand_{t-1})$
3. $demand_{t} = demand_{t-1}$

In all model, I drop observations where the outcome variable or the independent variables are missing i.e. no imputation.

In [7]:
from sklearn.model_selection import TimeSeriesSplit, GroupKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error as rmsle

In [8]:
# cross-validation for linear regression
# y = adjusted demand
# x = lagged adjusted demand

gap = 1
min_week = train['Week_num'].min()
max_week = train['Week_num'].max()

n_folds = 4
n_models = 3
i = 0
model_mse = np.zeros(shape=(n_folds, n_models))
model_msle = np.zeros(shape=(n_folds, n_models))
lr = LinearRegression()
lr2 = LinearRegression()

for week in range(min_week + gap + 1, max_week):

    # model 1
    train_wo_na = train.dropna(subset=['adj_demand_1', 'adjusted_demand'])
    df_tt = train_wo_na[train_wo_na['Week_num'] < week]
    df_ho = train_wo_na[train_wo_na['Week_num'] == week]

    lr.fit(X=df_tt[['adj_demand_1']], y=df_tt['adjusted_demand'])
    pred = lr.predict(X=df_ho[['adj_demand_1']])

    model_mse[i, 0] = root_mean_squared_error(y_true = df_ho['adjusted_demand'], y_pred = pred)
    model_msle[i, 0] = rmsle(y_true = df_ho['adjusted_demand'], y_pred = pred)

    # model 2
    train_wo_na = train.dropna(subset=['y', 'y_1'])

    df_tt = train_wo_na[train_wo_na['Week_num'] < week]
    df_ho = train_wo_na[train_wo_na['Week_num'] == week]
    lr2.fit(X=df_tt[['y_1']], y=df_tt['y'])
    pred = np.exp(lr2.predict(X=df_ho[['y_1']]))

    model_mse[i, 1] = root_mean_squared_error(y_true = np.exp(df_ho['y']), y_pred = pred)
    model_msle[i, 1] = rmsle(y_true = np.exp(df_ho['y']), y_pred = pred)

    #model 3
    train_wo_na = train.dropna(subset=['adj_demand_1', 'adjusted_demand'])
    df_tt = train_wo_na[train_wo_na['Week_num'] < week]
    df_ho = train_wo_na[train_wo_na['Week_num'] == week]
    pred = df_ho['adj_demand_1']

    model_mse[i, 2] = root_mean_squared_error(y_true = df_ho['adjusted_demand'], y_pred = pred)
    model_msle[i, 2] = rmsle(y_true = df_ho['adjusted_demand'], y_pred = pred)

    i += 1
print(model_mse.mean(axis=0))  
print(model_msle.mean(axis=0)) 


[13.28691466 34.37158866 13.45602846]
[0.55621516 0.51196662 0.56266535]


* We can expand this model to include missing clients
* We can include longer lagged in the model
* Auto ARIMA i.e. find out the right number of lags
* We can use the average of the client's observations for prediction
* Calculate autocorrelation
* XGBoost

## Out-of-Sample Prediction
One of the main challenges is to predict the demand for the following cases:
1. Existing Clients, New Products
2. New Clients, Exisiting Products
3. New Clients, New Prodcuts

In [9]:
# load train.csv
data_path = "..\product-inventory"
filename = os.path.join(data_path, "grupo-bimbo-inventory-demand/train.csv.zip")

train = pd.read_csv(filename, 
                 usecols=['Semana', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])

# rename columns
train = train.rename(columns={  'Semana': 'Week_num',
                                'Cliente_ID': 'Client_ID',
                                'Demanda_uni_equil': 'adjusted_demand',
                                'Producto_ID': 'Product_ID'})

# load test.csv
data_path = "..\product-inventory"
filename = os.path.join(data_path, "grupo-bimbo-inventory-demand/test.csv.zip")

test = pd.read_csv(filename, 
                 usecols=['Semana', 'Producto_ID', 'Cliente_ID'])

# rename columns
test = test.rename(columns={  'Semana': 'Week_num',
                                'Cliente_ID': 'Client_ID',
                                'Producto_ID': 'Product_ID'})


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:15: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:15: SyntaxWarning: invalid escape sequence '\p'
C:\Users\amirh\AppData\Local\Temp\ipykernel_19872\76495832.py:2: SyntaxWarning: invalid escape sequence '\p'
  data_path = "..\product-inventory"
C:\Users\amirh\AppData\Local\Temp\ipykernel_19872\76495832.py:15: SyntaxWarning: invalid escape sequence '\p'
  data_path = "..\product-inventory"


The list of existing clients in the test data.

In [33]:
# list of exisiting and new clients
testID = test['Client_ID'].unique().tolist()
trainID = train['Client_ID'].unique().tolist()
commonID = list(set(testID).intersection(set(trainID)))
newID = list(set(testID) - set(trainID))

print(len(newID)/len(test['Client_ID'].unique()))

print(len(test['Client_ID'].unique()))

0.012967615182698037
745164


The list of existing and new products in the test data

In [11]:
# list of existing and new products
testPID = test['Product_ID'].unique().tolist()
trainPID = train['Product_ID'].unique().tolist()
commonPID = list(set(testPID).intersection(set(trainPID)))
newPID = list(set(testPID) - set(trainPID))

print(len(newPID)/len(test['Product_ID'].unique()))

print(len(test['Product_ID'].unique()))

0.022339027595269383
1522


### 1. New Clients, Existing Products

In [18]:
# identify the list of existing products with new clients in the test data
existingPID = test.loc[(test['Client_ID'].isin(newID)) & 
                       (test['Product_ID'].isin(commonPID)),
                       'Product_ID'].unique().tolist()

# use product's average demand in week 3-9 as a prediction for the new client.
#pred_1 contains existing product with new client, and a prediction for the client's demand.
pred_1 = train.loc[train['Product_ID'].isin(existingPID)].groupby('Product_ID', as_index=False).agg({'adjusted_demand': 'mean'})
test = test.merge(right=pred_1, how='left', on='Product_ID')
test.loc[test['Client_ID'].isin(commonID), 'adjusted_demand'] = np.nan

del pred_1
test

,Week_num,Client_ID,Product_ID,adjusted_demand
0,11,4639078,35305,NaN
1,11,4705135,1238,NaN
2,10,4549769,32940,NaN
3,11,4717855,43066,NaN
4,11,966351,1277,NaN
...,...,...,...,...
6999246,11,4379638,1232,NaN
6999247,10,970421,43069,NaN
6999248,11,192749,30532,NaN
6999249,11,286071,35107,NaN


### 2. Existing Client, New Products

In [19]:
# identify the list of existing clients with new products in the test data
existingID = test.loc[(test['Client_ID'].isin(commonID)) & 
                      (test['Product_ID'].isin(newPID)),
                      'Client_ID'].unique().tolist()

# use client's average demand in week 3-9 as a prediction for new product.
#pred_1 contains existing clients with new products, and a prediction for the product.
pred_1 = train.loc[train['Client_ID'].isin(existingID)].groupby('Client_ID', as_index=False).agg({'adjusted_demand': 'mean'})
test = test.merge(right=pred_1, how='left', on='Client_ID')
test.loc[test['Product_ID'].isin(commonPID), 'adjusted_demand_y'] = np.nan


del pred_1
test
# WATCH OUT: This replaces missing values for existing clients and existing products with the client's average demand. 
# THEY SHOULD BE REPLACED WITH THE ACTUAL PREDICTION

,Week_num,Client_ID,Product_ID,adjusted_demand_x,adjusted_demand_y
0,11,4639078,35305,NaN,NaN
1,11,4705135,1238,NaN,NaN
2,10,4549769,32940,NaN,NaN
3,11,4717855,43066,NaN,NaN
4,11,966351,1277,NaN,NaN
...,...,...,...,...,...
6999246,11,4379638,1232,NaN,NaN
6999247,10,970421,43069,NaN,NaN
6999248,11,192749,30532,NaN,NaN
6999249,11,286071,35107,NaN,NaN


### 3. New Clients, New Products
The intersection of new clients and new product in the test data. Here the first guess is the average demand for all product across all weeks.


In [20]:
pred_1 = train['adjusted_demand'].mean()
test.loc[(test['Product_ID'].isin(newPID)) & (test['Client_ID'].isin(newID)), 'adjusted_demand'] = pred_1
test

,Week_num,Client_ID,Product_ID,adjusted_demand_x,adjusted_demand_y,adjusted_demand
0,11,4639078,35305,NaN,NaN,NaN
1,11,4705135,1238,NaN,NaN,NaN
2,10,4549769,32940,NaN,NaN,NaN
3,11,4717855,43066,NaN,NaN,NaN
4,11,966351,1277,NaN,NaN,NaN
...,...,...,...,...,...,...
6999246,11,4379638,1232,NaN,NaN,NaN
6999247,10,970421,43069,NaN,NaN,NaN
6999248,11,192749,30532,NaN,NaN,NaN
6999249,11,286071,35107,NaN,NaN,NaN


What share of the test dataset is estimated?

In [35]:
print(len(test['adjusted_demand']))
      
1-test[['adjusted_demand_x', 'adjusted_demand_y', 'adjusted_demand']].isna().mean()

6999251


adjusted_demand_x    0.004277
adjusted_demand_y    0.003646
adjusted_demand      0.007940
dtype: float64